<a href="https://colab.research.google.com/github/LoQiseaking69/Jacobian-calculator/blob/main/Jacobian-Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sympy import symbols, parse_expr, lambdify, Matrix, simplify
import ipywidgets as widgets
from IPython.display import display, clear_output

class JacobianComputation:
    def __init__(self, function_expressions, variable_names):
        self.variable_names = variable_names
        self.functions = [parse_expr(func, evaluate=False) for func in function_expressions]
        self.variables = symbols(variable_names)
        self.jacobian = Matrix(self.functions).jacobian(self.variables)

    def compute_jacobian(self):
        return self.jacobian

    def evaluate_jacobian(self, variable_values):
        jacobian_func = lambdify(self.variables, self.jacobian, modules=["numpy"])
        return np.array(jacobian_func(*variable_values))

    def plot_jacobian(self, jacobian_matrix):
        plt.figure(figsize=(10, 8))
        sns.heatmap(jacobian_matrix, annot=True, cmap='viridis', fmt='.2f')
        plt.title('Jacobian Matrix Visualization')
        plt.xlabel('Variables')
        plt.ylabel('Functions')
        plt.show()

    def jacobian_determinant(self):
        return simplify(self.jacobian.det())

# Widgets for input
function_input = widgets.Textarea(
    value='l1*cos(theta1) + l2*cos(theta1 + theta2); l1*sin(theta1) + l2*sin(theta1 + theta2)',
    description='Functions:',
    disabled=False,
    layout=widgets.Layout(width='95%', height='100px')
)

variable_input = widgets.Text(
    value='l1, l2, theta1, theta2',
    description='Variables:',
    disabled=False
)

value_input = widgets.Text(
    value='1, 1, np.pi/4, np.pi/4',
    description='Values:',
    disabled=False
)

evaluate_button = widgets.Button(
    description='Evaluate Jacobian',
    disabled=False,
    button_style=''
)

output = widgets.Output()

def on_evaluate_button_clicked(b):
    with output:
        clear_output(wait=True)
        try:
            # Process variable names
            variable_names = [var.strip() for var in variable_input.value.split(',')]
            # Process function strings
            function_strings = [func.strip() for func in function_input.value.split(';')]
            # Parse values, allowing for numpy constants
            variable_values = eval(f'np.array([{value_input.value}])', {"np": np})

            jc = JacobianComputation(function_strings, variable_names)

            # Check if numerical values have been provided for all variables
            if all(isinstance(v, (int, float, complex, np.number)) for v in variable_values):
                evaluated_jacobian = jc.evaluate_jacobian(variable_values)
                print("Evaluated Jacobian matrix at the points:", dict(zip(variable_names, variable_values)))
                print(evaluated_jacobian)
                jc.plot_jacobian(evaluated_jacobian)
            else:
                # Handle symbolic case
                print("Jacobian matrix (symbolic):")
                print(jc.compute_jacobian())
                print("\nJacobian Determinant (symbolic):", jc.jacobian_determinant())

        except Exception as e:
            print(f"An error occurred: {e}")

evaluate_button.on_click(on_evaluate_button_clicked)
display(function_input, variable_input, value_input, evaluate_button, output)

Textarea(value='l1*cos(theta1) + l2*cos(theta1 + theta2); l1*sin(theta1) + l2*sin(theta1 + theta2)', descripti…

Text(value='l1, l2, theta1, theta2', description='Variables:')

Text(value='1, 1, np.pi/4, np.pi/4', description='Values:')

Button(description='Evaluate Jacobian', style=ButtonStyle())

Output()